In [21]:
import emip_toolkit as emtk
import filter_fixation as ff
import numpy as np
import pandas as pd
import json

# %matplotlib tk      
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def draw(centers):
    fixs = []
    for data in centers.values():
        if data[0] == -1 and data[1] == -1: continue
        else:
            x, y, duration = data
            fixs.append([x, y])
    fixs = np.array(fixs)
    l = plt.plot(fixs[:, 0], fixs[:, 1], alpha=0.4, c='orange')
    line = l.pop(0)
    scatter = plt.scatter(fixs[:, 0], fixs[:, 1], alpha=0.5, s=100, c='yellow')
    return line, scatter

def is_fixation(x, y, centers):
    if find_index(centers):
        # Empty space exists, new fixation needs to be drawn
        return None
    for index, (c_x, c_y, duration) in centers.items():
        # If click is on one fixation -> Hide it
        if c_x-12 < x < c_x+12 and c_y-12 < y < c_y+12:
            return index
    return None

def find_index(centers):
    for index, data in centers.items():
        if data[0] == -1 and data[1] == -1:
            return index
    return None

In [50]:
fig = plt.figure(figsize=(12,8))

path = 'datasets/GazeBase_TEX/Data/S_8331_S1_TEX.csv'

pixel_data = ff.process_GazeBase(path)
fixations, saccades = ff.filter_fixations(pixel_data)

path = path.split('/')[-1]
trial_id = int(path[8])
round_id = path[2]
participant_id = path[3:6]
imagepath = f"datasets/GazeBase_TEX/StimuliExamples/TEXScreenshots/TEX_R{round_id}S{trial_id}_bg.png"

img = mpimg.imread(imagepath)

# Initialize fixation centers
centers = {}
for index, fixation in fixations.items():
    fixation = fixation.get_fixation()
    duration = fixation[3]
    if duration < 90:
        continue
    x = fixation[4]
    y = fixation[5]
    centers[index] = [x, y, duration]

line, scatter = draw(centers)

def onclick(event):
    x, y = event.xdata, event.ydata
    global line, scatter, centers
    index = is_fixation(x, y, centers)

    if index:
        # If click is on one fixation -> Hide it
        centers[index][0] = -1
        centers[index][1] = -1
    else:
        # If click is on empty spot -> Find empty spot and fill with current position
        index = find_index(centers)
        if index:
            centers[index][0] = x
            centers[index][1] = y

    line.remove()
    scatter.remove()
    
    line, scatter = draw(centers)
    fig.canvas.draw()
    
    with open(f'corrections/R{round_id}S{trial_id}_{participant_id}.txt', 'w') as file:
        file.write(json.dumps(centers))

cid = fig.canvas.mpl_connect('button_press_event', onclick)
imgplot = plt.imshow(img)
plt.show()

In [20]:
centers

{1: [289.2018739445997, 233.63812461803616, 160],
 3: [782.2182261948201, 476.8571456625471, 203],
 5: [768.8265728349685, 43.40699265887566, 143],
 7: [611.9875030405026, 75.78988152270499, 168],
 9: [656.0351820106783, 62.30370146779724, 162],
 11: [771.8578698582584, 61.256480724849176, 216],
 13: [812.4183808210983, 60.18896954255697, 136],
 15: [895.9117683382177, 61.34771254466603, 153],
 17: [992.1879505091508, 55.986140853410824, 166],
 21: [1064.7708285834603, 53.86596843675651, 144],
 23: [1116.4151775891996, 56.94731967337547, 112],
 27: [751.0631137521743, 95.1340182924285, 244],
 29: [902.9525586175982, 95.06325631055508, 215],
 31: [983.8638170148344, 89.21447362694018, 152],
 33: [610.8281566020122, 145.57184490891433, 103],
 35: [606.2669326308568, 148.37338975044275, 124],
 37: [661.3543478493176, 144.71793385944596, 184],
 39: [753.3999996689605, 140.35395769495076, 202],
 41: [816.9301893578918, 137.30188505243694, 159],
 45: [887.7308187447533, 133.2647791739748, 15

In [1]:
import json

In [2]:
import os

In [7]:
filenames = ['R8S1_192.txt', 'R8S1_179.txt', 'R8S1_152.txt', 'R8S1_180.txt', 'R8S1_154.txt', 'R8S1_196.txt', 'R8S1_244.txt', 'R8S1_331.txt', 'R8S1_285.txt', 'R8S1_120.txt', 'R8S1_043.txt', 'R8S1_137.txt', 'R8S1_136.txt', 'R8S1_259.txt', 'R8S1_271.txt', 'R8S1_107.txt', 'R8S1_064.txt', 'R8S1_288.txt', 'R8S1_128.txt', 'R8S1_314.txt', 'R8S1_249.txt', 'R8S1_063.txt', 'R8S1_207.txt', 'R8S1_007.txt', 'R8S1_016.txt', 'R8S1_215.txt', 'R8S1_001.txt', 'R8S1_217.txt', 'R8S1_216.txt']

In [15]:
for file in filenames:
    with open('corrections/' + file) as correction:
        cor = json.loads(correction.read())
    participant_id = file.split('.')[0]
    with open('originals/' + participant_id + '_ORIGINAL.json') as original:
        ori = json.loads(original.read())
        
        print(file, len(cor) == len(ori))

R8S1_192.txt True
R8S1_179.txt True
R8S1_152.txt True
R8S1_180.txt True
R8S1_154.txt True
R8S1_196.txt True
R8S1_244.txt True
R8S1_331.txt True
R8S1_285.txt True
R8S1_120.txt True
R8S1_043.txt False
R8S1_137.txt True
R8S1_136.txt True
R8S1_259.txt True
R8S1_271.txt True
R8S1_107.txt True
R8S1_064.txt True
R8S1_288.txt True
R8S1_128.txt True
R8S1_314.txt True
R8S1_249.txt True
R8S1_063.txt True
R8S1_207.txt True
R8S1_007.txt False
R8S1_016.txt False
R8S1_215.txt True
R8S1_001.txt False
R8S1_217.txt True
R8S1_216.txt True


In [27]:
for file in filenames:
    print(file)
    participant = file.split('.')[0][-3:]
    path = 'datasets/GazeBase_TEX/Data/S_8' + participant + '_S1_TEX.csv'

    pixel_data = ff.process_GazeBase(path)
    fixations, saccades = ff.filter_fixations(pixel_data)

    path = path.split('/')[-1]
    trial_id = int(path[8])
    round_id = path[2]
    participant_id = path[3:6]
    imagepath = f"datasets/GazeBase_TEX/StimuliExamples/TEXScreenshots/TEX_R{round_id}S{trial_id}_bg.png"


# Initialize fixation centers
    centers = {}
    for index, fixation in fixations.items():
        fixation = fixation.get_fixation()
        duration = fixation[3]
        if duration < 90:
            continue
        x = fixation[4]
        y = fixation[5]
        centers[index] = [x, y, duration]
    
    with open('ori/R8S1_' + participant + '_ORIGINAL.json', 'w') as output:
        output.write(json.dumps(centers))

R8S1_192.txt
R8S1_179.txt
R8S1_152.txt
R8S1_180.txt
R8S1_154.txt
R8S1_196.txt
R8S1_244.txt
R8S1_331.txt
R8S1_285.txt
R8S1_120.txt
R8S1_043.txt
R8S1_137.txt
R8S1_136.txt
R8S1_259.txt
R8S1_271.txt
R8S1_107.txt
R8S1_064.txt
R8S1_288.txt
R8S1_128.txt
R8S1_314.txt
R8S1_249.txt
R8S1_063.txt
R8S1_207.txt
R8S1_007.txt
R8S1_016.txt
R8S1_215.txt
R8S1_001.txt
R8S1_217.txt
R8S1_216.txt


In [37]:
for file in filenames:
    with open('corrections/' + file) as correction:
        cor = json.loads(correction.read())
    participant_id = file.split('.')[0]
    with open('ori/' + participant_id + '_ORIGINAL.json') as original:
        ori = json.loads(original.read())
        
        print(file, len(cor) == len(ori))

R8S1_192.txt True
R8S1_179.txt True
R8S1_152.txt True
R8S1_180.txt True
R8S1_154.txt True
R8S1_196.txt True
R8S1_244.txt True
R8S1_331.txt True
R8S1_285.txt True
R8S1_120.txt True
R8S1_043.txt False
R8S1_137.txt True
R8S1_136.txt True
R8S1_259.txt True
R8S1_271.txt True
R8S1_107.txt True
R8S1_064.txt True
R8S1_288.txt True
R8S1_128.txt True
R8S1_314.txt True
R8S1_249.txt True
R8S1_063.txt True
R8S1_207.txt True
R8S1_007.txt False
R8S1_016.txt False
R8S1_215.txt True
R8S1_001.txt False
R8S1_217.txt True
R8S1_216.txt True


In [48]:
for file in filenames:
    new_output = {}
    with open('corrections/' + file) as correction:
        cor = json.loads(correction.read())
    for key in cor:
        if cor[key][2] >= 90:
            new_output[key] = cor[key]
    with open('ori/' + file.replace('.txt', '') + '_CORRECTED.json', 'w') as output:
        output.write(json.dumps(new_output))

In [50]:
for file in filenames:
    with open('ori/' + file.replace('.txt', '_CORRECTED.json')) as correction:
        cor = json.loads(correction.read())
    participant_id = file.split('.')[0]
    with open('originals/' + participant_id + '_ORIGINAL.json') as original:
        ori = json.loads(original.read())
        
        print(file, len(cor), len(ori))

R8S1_192.txt 264 264
R8S1_179.txt 246 246
R8S1_152.txt 233 233
R8S1_180.txt 271 271
R8S1_154.txt 211 211
R8S1_196.txt 199 199
R8S1_244.txt 245 245
R8S1_331.txt 215 215
R8S1_285.txt 250 250
R8S1_120.txt 217 217
R8S1_043.txt 219 219
R8S1_137.txt 118 118
R8S1_136.txt 139 139
R8S1_259.txt 224 224
R8S1_271.txt 189 189
R8S1_107.txt 214 214
R8S1_064.txt 189 189
R8S1_288.txt 257 257
R8S1_128.txt 217 217
R8S1_314.txt 255 255
R8S1_249.txt 232 232
R8S1_063.txt 218 218
R8S1_207.txt 224 224
R8S1_007.txt 229 229
R8S1_016.txt 186 186
R8S1_215.txt 229 229
R8S1_001.txt 256 256
R8S1_217.txt 229 229
R8S1_216.txt 142 142


In [34]:
len(ori)

219